比较复杂的语言：JavaScript,R<br>
不用做处理的语言：HTML<br>
版本不一致的语言：SQL(本人下载的是sqlite)


In [2]:
#支持的语言
supported_languages=['c',
 'cpp',
 'c_sharp',
 'go',
 'java',
 'php',
 'python',
 'ruby',
 'rust',
 'tsx',
 'fortran',
 'kotlin',
 'cuda',
 'scala']
#该字典为不同语言的主体节点
dict_codebody={'c': ['function_definition', 'struct_specifier'],
 'cpp': ['function_definition', 'class_specifier', 'template_declaration', 'struct_specifier'], 
 'c_sharp': ['class_declaration', 'interface_declaration', 'struct_declaration'], 
 'go': ['type_declaration', 'method_declaration', 'function_declaration'], 
 'java': ['class_declaration', 'interface_declaration'], 
 'php': ['class_declaration', 'expression_statement', 'interface_declaration', 'function_definition'], 
 'python': ['function_definition', 'class_definition'], 'ruby': ['class', 'method'],
  'rust': ['trait_item', 'struct_item',  'enum_item', 'function_item', 'type_item'],
   'tsx': ['lexical_declaration', 'export_statement', 'expression_statement', 'function_declaration', 'class_declaration', 'interface_declaration'], 
   'fortran': ['module', 'subroutine', 'function', 'submodule'], 
   'kotlin': ['call_expression', 'function_declaration', 'prefix_expression', 'class_declaration', 'object_declaration'], 
   'cuda': ['function_definition', 'template_declaration', 'struct_specifier', 'class_specifier', 'type_definition'], 
   'scala': ['class_definition', 'object_definition', 'function_definition', 'trait_definition']}
#该字典为不同语言需要向下进一层的节点
dict_next={'c': [],
 'cpp': ['namespace_definition','declaration_list','namespace_declaration'], 
 'c_sharp': ['namespace_definition','declaration_list','namespace_declaration'], 
 'go': [], 
 'java': [], 
 'php': [], 
 'python': [], 'ruby': ['module'],
  'rust': ['impl_item','mod_item','declaration_list'],
   'tsx': [], 
   'fortran': ['program'], 
   'kotlin': [], 
   'cuda': ['namespace_definition','declaration_list','namespace_declaration'], 
   'scala': []}
#该字典为不同语言主体的前缀，需要在body的前面作为提示信息
'''
dict_codehint={'c': ['comment', ' type_definition', 'declaration'], 
'cpp': ['comment', 'type_definition', 'declaration'], 'c_sharp': ['comment', 'global_statement'], 
'go': ['comment', 'var_declaration'], 'java': ['block_comment', 'line_comment'], 
'php': ['php_tag', 'namespace_definition', 'namespace_use_declaration', 'comment'],
 'python': ['comment', 'expression_statement-string','expression_statement'], 'ruby': ['comment'], 
 'rust': ['line_comment', 'block_comment'], 'tsx': ['comment'], 'fortran': ['comment'], 
 'kotlin': ['comment'], 'cuda': ['comment'], 'scala': ['comment']}'''
dict_codehint={'c': ['comment'], 
'cpp': ['comment'], 'c_sharp': ['comment'], 
'go': ['comment'], 'java': ['block_comment', 'line_comment'], 
'php': [ 'comment'],
 'python': ['comment''expression_statement'], 'ruby': ['comment'], 
 'rust': ['line_comment', 'block_comment'], 'tsx': ['comment'], 'fortran': ['comment'], 
 'kotlin': ['comment'], 'cuda': ['comment'], 'scala': ['comment']}

#映射字典
dict_codegeex2ts={
 'c': 'c',
 'c++': 'cpp',
 'cpp': 'cpp',
 'c#': 'c_sharp',
 'csharp': 'c_sharp',
 'c-sharp':'c_sharp',
 'css': 'css',
 'cuda': 'cuda',
 'dart': 'dart',
 'lua': 'lua',
 'objectivec': 'objc',
 'objective-c': 'objc',
 'python': 'python',
 'perl': 'perl',
 'prolog': 'prolog',
 'lisp': 'elisp',
 'java': 'java',
 'scala': 'scala',
 'tex': 'latex',
 'html': 'html',
 'php': 'php',
 'js': 'javascript',
 'javascript': 'javascript',
 'typescript': 'tsx',
 'go': 'go',
 'rust': 'rust',
 'sql': 'sqlite',
 'kotlin': 'kotlin',
 'ruby': 'ruby',
 'pascal': 'pascal',
 'r': 'r',
 'fortran': 'fortran',
 'matlab': 'matlab'}



In [3]:
from tree_sitter import Language, Parser
import tree_sitter
print(tree_sitter.__package__)
#解析代码获取parsetree
def getast_tree(language,text):
    LANGUAGE=Language('build/my-languages.so', language)
    parser=Parser()
    parser.set_language(LANGUAGE)
    tree=parser.parse(bytes(text,"utf8"))
    return tree
def search(node,language):
    nodes=set()
    node_childs = node.children
    if node_childs==[]:
        return set()
    for i in node_childs:
        nodes.add(i.type)
        nodes=nodes|search(i,language)
    return nodes
def test_text(language,text):
    tree=getast_tree(language,text)
    nodes=set()
    nodes=search(tree.root_node,language) 
    for i in nodes:
        if 'comment' in i:
            return True
    return False  
#对根节点的第一层节点进行搜索，将class、function节点加入
def dfs(node,language):
        node_childs = node.children
        if node_childs == []: return
        method_list=[]
        mod=""
        for i in range(len(node_childs)-1,-1,-1):
            if node_childs[i].type in dict_codebody[language]:
                method_list.append(node_childs[i])
                mod='body'
            elif node_childs[i].type in dict_next[language]:
                if type(dfs(node_childs[i],language))==list:
                    for j in dfs(node_childs[i],language):
                        method_list.append(j)
            elif node_childs[i].type in dict_codehint[language]:
                if mod=='body':
                    method_list.append(node_childs[i])
            else:
                mod='trash'
        return method_list

def split_code(language,code):
    "返回所有函数和函数节点信息"
    tree=getast_tree(language,code)
    method_nodes=[]
    method_nodes=dfs(tree.root_node,language)
    code_list = code.split('\n')
    methods_list = []
    method=""
    for n in range(len(method_nodes)-1,-1,-1):
        if method_nodes[n].type in dict_codehint[language]:
            method+=method_nodes[n].text.decode('utf-8') # bytes转str
            method+='\n'
        else:
            method+= method_nodes[n].text.decode('utf-8')
            if test_text(language,method):
                methods_list.append(method)
            method=""
    return methods_list


tree_sitter


### 对提取出的代码段进行检查，检查其是否有前注释以及注释是否过长

In [4]:

ThresholdOfLength=1
ThresholdOfSymbol=20
#检查注释是否无意义
def CheckSymbol(line):
     unique_chars = set(line)
     count = len(unique_chars)
     ratio=len(line)/count
     if ratio>ThresholdOfSymbol:
          print(line,'\n')
          print(ratio,'\n')
          return False
     else:
          return True
def cutoff(node,language,threshold):
        node_childs = node.children
        if node_childs == []: return
        method_list=[]
        count=0
        for i in range(len(node_childs)):
            if node_childs[i].type in dict_codehint[language]:
                count+=1
                if count<=threshold:
                    method_list.append(node_childs[i])
            else:
                 method_list.append(node_childs[i])
        return method_list
def Pass(node,language,minbash):
        node_childs = node.children
        count=0
        method_list=[]
        for i in range(len(node_childs)):
             if node_childs[i].type in dict_codehint[language]:
                if CheckSymbol(node_childs[i].text.decode('utf-8'))==True:
                    method_list.append(node_childs[i])
                    count+=1
                if count>minbash:
                     return []
             else:
                  method_list.append(node_childs[i])
             
        return method_list
def cutoff(node,language,ThresholdOfLength):
        node_childs = node.children
        if node_childs == []: return
        method_list=[]
        count=0
        for i in range(len(node_childs)):
            if node_childs[i].type in dict_codehint[language]:
                if CheckSymbol(node_childs[i].text.decode('utf-8'))==True:
                    count+=1
                    if count<=ThresholdOfLength:
                         method_list.append(node_childs[i])
            else:
                 method_list.append(node_childs[i])
        return method_list
#检查前边是否有注释，如果没有的话将内部注释提取出来
def CheckComment(node,language,code):
     node_childs = node.children
     count=0
     for i in range(len(node_childs)):
          if node_childs[i].type in dict_codehint[language]:
               count+=1
     if count==0:
          print(code,'\n\n\n\n')
          Node=node_childs[-1].children
          comment=""
          print(Node)
          for i in range(len(Node)):
               if Node[i].type in dict_codehint[language]:
                    comment+=Node[i].text.decode('utf-8')
          method=node_childs[-1].text.decode('utf-8')
          print(comment,'\n\n\n')
          return comment+method
     else:
          return code

#检查注释长度                    
def CheckLength(language,code,mod):
    "返回所有函数和函数节点信息"
    tree=getast_tree(language,code)
    method_nodes=[]
    if mod=='cutoff':
          method_nodes=cutoff(tree.root_node,language,ThresholdOfLength)
          method=""
          for n in range(len(method_nodes)):
               method+=method_nodes[n].text.decode('utf-8') # bytes转str
               method+='\n'
          return method
    elif mod=='pass':
         method_nodes=Pass(tree.root_node,language,ThresholdOfLength)
         method=""
         for n in range(len(method_nodes)):
               method+=method_nodes[n].text.decode('utf-8') # bytes转str
               method+='\n'
         return method


### 测试文件


In [5]:
import json
import os
file='text.txt'
text=[]
language='c++'
print(language+'\n')
with open(file,'r') as f:
	text=f.read()
#分割代码
methods_list=[]
methods_list.append(split_code(dict_codegeex2ts[language],text))
with open("C:/Users/liuyi/Desktop/ast/text3.txt","w")as f:
	for i in methods_list:
		for j in i:
			j=CheckLength(dict_codegeex2ts[language],j,'pass')

c++

//----------------------------------------------------------------------------- 

39.5 

// --------------------------------------------------------------- 

22.0 

//===----------------------------------------------------------------------===// 

26.666666666666668 

//===----------------------------------------------------------------------===// 

26.666666666666668 

//===----------------------------------------------------------------------===// 

26.666666666666668 

//===----------------------------------------------------------------------===// 

26.666666666666668 

//===----------------------------------------------------------------------===// 

26.666666666666668 

//===----------------------------------------------------------------------===// 

26.666666666666668 

//===----------------------------------------------------------------------===// 

26.666666666666668 

//===----------------------------------------------------------------------===// 

26.666666666666668 

In [74]:
import random
#c,cpp,csharp,go,php
dictof={'template_declaration':'','type_declaration':'',' namespace_definition':'declaration_list',
        'function_definition':'compound_statement','expression_statement':'',
        'class_specifier':'field_declaration_list','class_declaration':'declaration_list',
        'interface_declaration':'declaration_list','struct_declaration':'declaration_list',
        'struct_specifier':'field_declaration_list','function_declaration':'block',
        'method_declaration':'block',}
#java,python
dictof1={'interface_declaration':'interface_body','class_declaration':'class_body','class_difinition':'block','function_difinition':'block',}
#ruby无节点
#代码填空
def GapFilling(code,language):
     tree=getast_tree(language,code)
     node_childs = tree.root_node.children
     codelist=[]
     keynode=""
     leftcode=[]
     isfind=False
     for index in node_childs:
          if index.type in dict_codebody[language]:
               isfind=True
               j=index.children
               key=index.type
               for i in j:
                    if i.type!=dictof[key]:
                         codelist.append(i)
                    else:
                         node_childs=i.children
                         index=random.randint(1,len(node_childs)-2)
                         for k in range(len(node_childs)):
                              if index == k:
                                   keynode=node_childs[k]
                              elif index > k:
                                   codelist.append(node_childs[k])
                              else:
                                   leftcode.append(node_childs[k])
          elif isfind==False and index.type not in dict_codebody[language]:
               codelist.append(index)
          else:
               leftcode.append(index)
          
     method=""
     for n in range(len(codelist)):
          method+=codelist[n].text.decode('utf-8') # bytes转str
          method+='\n'
     method1=""
     for n in range(len(leftcode)):
          method1+=leftcode[n].text.decode('utf-8') # bytes转str
          method1+='\n'
     if keynode!="":
          keynode=keynode.text.decode('utf-8')
          print(method,'\n\n\n\n\n\n\n\n\n\n',keynode,'\n\n\n\n\n\n\n\n\n',method1)
          #return method,keynode,method1
with open('./gapfilling.txt','r')as f:
     code=f.read()
     GapFilling(code,'cpp')